In [ ]:
import datetime
def getDate(timestamp):
    date = datetime.datetime.fromtimestamp(int(timestamp) / 1000.0)
    date = date.strftime('%Y-%m-%d %H:%M:%S')
    return date

def getTime(date):
    A = date.split()[1]
    A = A.split(":")[0]
    return A

In [ ]:
import string
import json
import nltk;

import spacy
nlp = spacy.load('en')

In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
# nltk.download('punkt')

stop_words = set(stopwords.words('english')) 

In [ ]:

tweets = json.load(open("gg2013.json"))

In [144]:
import re

gg_stop_words = ['Globe', 'RT', 'http', 'Golden', 'Globes', 'GoldenGlobes', 'Goldenglobes', 'Goldenglobe', 'gg','golden globes', 'golden globe', 'goldenglobe','goldenglobes','gg2015','gg15','goldenglobe2015','goldenglobe15','goldenglobes2015','goldenglobes15', 'gg2013','gg13','goldenglobe2013','goldenglobe13','goldenglobes2013','goldenglobes13', 'rt', '2013', '2015', '...', '`', 'MTVNews']

gg_stop_words.extend(stop_words)

clean_tweets = []

for tweet in tweets:
    text = tweet['text']
    text_tokens = word_tokenize(text)
    t = []
    sent = ''
    for w in text_tokens:
        if w not in gg_stop_words:
            if w not in string.punctuation: 
                t.append(w)
    for i in range(len(t)): 
        sent += t[i] + ' '
    clean_tweets.append(sent)



In [ ]:
clean_tweets

In [ ]:
# def get_winner(award_name):
    
#     search_term = award_name.lower()
    
# #     if award_name in people_award.keys():
# #         search_term = people_award.get(award_name)
# #     else:
# #         search_term = film_award.get(award_name)
# #     print(search_term)
    
#     relevant_tweets = []
#     for t in clean_tweets:
#         text = t.lower()
#         if re.search(search_term, text) and 'RT' not in text:
#             relevant_tweets.append(text) 
            
#     possible_answer = []
            
#     for t in relevant_tweets:
#         for w in t:
            
#         text = t.lower()
#         tokenized_t = text.split()
#         if 'wins' in tokenized_t:
#             index = tokenized_t.index('wins')
#             before_win = []
#             string = tokenized_t[0]
#             for i in range(1, index): 
#                 string += ' ' + tokenized_t[i]
#             before_win.append(string)
#             for s in before_win:
#                 names = [ee for ee in nlp(s).ents if ee.label_ == 'PERSON']
#                 print(names)
#                 print('???????')
#                 possible_answer.append(names)
#         if 'goes to' in text:
#             index = text.find('goes to')
#             substring_index = index + 7
#             substring = text[substring_index:]
#             for w in substring:
#                 names = [ee for ee in nlp(w).ents if ee.label_ == 'PERSON']
#                 possible_answer.append(names)
            
            
# #     name_pair = []
# #     for possible_name_pair in possible_answer:
# #         if nltk.pos_tag(possible_name_pair)[0] == 'NNP' and nltk.pos_tag(possible_name_pair)[1] == 'NNP':
# #             name_pair.append(possible_name_pair)
        
    
# #     print(possible_answer)
#     #return relevant_tweets

In [ ]:
import spacy
en = spacy.load('en')

sents = en(open('input.txt').read())
people = [ee for ee in sents.ents if ee.label_ == 'PERSON']

In [145]:
def get_names(text):
    article = nlp(text)
    labels = [x.label_ for x in article.ents]
    [(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(text) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]
    parts_of_speech = dict([(str(x), x.label_) for x in nlp(text).ents])
    names = []
    for (key, value) in parts_of_speech.items() :
        if(value == "PERSON") :
            names.append(key)
    return names

In [154]:
people_award = {}

film_award = {}

film_award['best screenplay - motion picture'] = (r'[Bb]est [Ss]creenplay')

people_award['best director - motion picture'] = (r'[Bb]est [Dd]irector')

people_award['best performance by an actress in a television series - comedy or musical'] = (r'[Bb]est [Pp]erformance.*[Aa]ctress.*[Tt].*[Vv].* [Mm]usical')

film_award['best foreign language film'] = (r'[Ff]oreign [Ll]anguage [Ff]ilm')


people_award['best performance by an actor in a supporting role in a motion picture'] = (r'([Aa]ct[oe]r.*[Ss]upporting|[Ss]upporting [Aa]ct[oe]r).*(in a|in).*[Mm]otion.*')

people_award['best performance by an actress in a supporting role in a series, mini-series or motion picture made for television'] = (r'([Ss]upporting.*[Aa]ctress.*[Ss]eries|[Aa]ctress.*[Ss]upporting [Rr]ole.*[Ss]eries)')

film_award['best motion picture - comedy or musical'] = (r'[Bb]est [Mm]otion [Pp]icture .*[Co]medy.*[Mm]usical')

people_award['best performance by an actress in a motion picture - comedy or musical'] = (r'[Bb]est [Pp]erformance.*[Aa]ctress.*[Mm]otion [Pp]icture.*[Cc]omedy.*[Mu]sical')

film_award['best mini-series or motion picture made for television'] = (r'[Bb]est [Mm]ini.*[Ss]eries .*[Mm]otion [Pp]icture.*([Tt][Vv]|[Tt]elevision)')

film_award['best original score - motion picture '] = (r'[Bb]est [Oo]riginal [Ss]core.*[Mm]otion [Pp]icture')

people_award['best performance by an actress in a television series - drama'] = (r'[Bb]est.*[Aa]ctress.*([Tt][Vv]|[Tt]elevision).*[Dd]rama')

people_award['best performance by an actress in a motion picture - drama'] = (r'[Bb]est.*[Aa]ctress.*[Mm]otion [Pp]icture.*[Dd]rama')

people_award['cecil b. demille award'] = (r'[Cc]ecil.*[Dd]emille')

people_award['best performance by an actor in a motion picture - comedy or musical'] = (r'[Bb]est [Pp]erformance.* [Aa]ct[oe]r.*[Mm]otion [Pp]icture.*[Cc]omedy')

people_award['best performance by an actor in a supporting role in a series, mini-series or motion picture made for television'] = (r'([Aa]ctor.*[Ss]upporting|[Ss]upporting.*[Aa]ctor).*[Mm]ini')

people_award['best performance by an actress in a supporting role in a motion picture'] = (r'([Ss]upporting.*[Aa]ctress|[Aa]ctress.*[Ss]upporting).*([Mm]otion [Pp]icture|[Ff]ilm)')

film_award['best motion picture - drama'] = (r'([Bb]est|[Ll]eading) ((([Mm]otion)?\s?[Pp]icture|[Ff]ilm).*[Dd]rama|[Dd]rama.*(([Mm]otion)? [Pp]icture|[Ff]ilm))')

film_award['best television series - drama'] = (r'([Bb]est|[Ll]eading) (([Tt][Vv]|[Ss]eries).*[Dd]rama|([Tt][Vv]|[Ss]eries)|[Dd]rama [Ss]eries)')

people_award['best performance by an actor in a mini-series or motion picture made for television'] = (r'([Bb]est|[Ll]eading).*[Aa]ctor.*[Mm]ini.*[Tt].*[Vv]')

people_award['best performance by an actress in a mini-series or motion picture made for television'] = (r'([Bb]est|[Ll]eading).*[Aa]ctress.*[Mm]ini.*[Tt].*[Vv]')

film_award['best animated feature film'] = (r'[Bb]est [Aa]nimated')

film_award['best original song - motion picture'] = (r'[Bb]est ([Oo]riginal )?[Ss]ong')

people_award['best performance by an actor in a motion picture - drama'] = (r'[Aa]ct[oe]r.*(([Mm]otion)[Pp]icture|[Ff]ilm).*([Dd]rama)')

film_award['best television series - comedy or musical'] = (r'[Bb]est ([Tt][Vv]|[Tt]elevision).*[Cc]omedy')

people_award['best performance by an actor in a television series - drama'] = (r'[Aa]ct[oe]r( in a| in)?,? ([Tt][Vv]|[Tt]elevision).*[Dd]rama')

people_award['best performance by an actor in a television series - comedy or musical'] = (r'[Aa]ct[oe]r( in a| in)?,? ([Tt][Vv]|[Tt]elevision).*[Cc]omedy')

In [165]:
import operator

def get_winner(award_name):
    
    search_term = award_name
    if award_name in people_award.keys():
        search_term = people_award.get(award_name)
    elif award_name in film_award.keys():
        search_term = film_award.get(award_name)
#     print(search_term)
    revelant_tweets = []
    for tweet in clean_tweets: 
        if re.search(search_term, tweet):
            revelant_tweets.append(tweet)   
    count = {}
#     print(revelant_tweets)
    for tweet in revelant_tweets: 
        names = get_names(tweet)
        for name in names :
            if name in count :
                count[name] = count[name] + 1
            else:
                count[name] = 1
        winner = max(count.items(), key=operator.itemgetter(1))[0]
    return winner

In [166]:
# wrong answer (christoph waltz)

get_winner('best performance by an actor in a supporting role in a motion picture')


'Ed Harris'

In [167]:
# can't get the answer (jennifer lawrence)
# max() arg is an empty sequence

get_winner('best performance by an actress in a motion picture - comedy or musical')

ValueError: max() arg is an empty sequence

In [168]:
# can't get answer (Claire Danes)
# max() arg is an empty sequence

get_winner('best performance by an actress in a television series - drama')

ValueError: max() arg is an empty sequence

In [169]:
# can't get answer (Anne Hathaway)
# max() arg is an empty sequence

get_winner('best performance by an actress in a supporting role in a motion picture')

ValueError: max() arg is an empty sequence

In [170]:
# can't get answer (Damian Lewis)
# max() arg is an empty sequence

get_winner('best performance by an actor in a television series - drama')

ValueError: max() arg is an empty sequence

In [171]:
# can't get answer (Don Cheadle)
# max() arg is an empty sequence

get_winner('best performance by an actor in a television series - comedy or musical')

ValueError: max() arg is an empty sequence